In [1]:
# Standard library imports
import os
from pathlib import Path
import sys  

ROOT_DIR = Path().resolve().parents[1]
sys.path.append(str(ROOT_DIR))

# Third-party imports
import pandas as pd
import numpy as np

# Local application imports
from config.paths import RAW_MARKET_FILE


In [2]:
ds = pd.read_csv(RAW_MARKET_FILE)

In [3]:
ds.describe()

,Price,Open,High,Low,Vol.,Change %
count,57969.000000,57969.000000,57969.000000,57969.000000,5.794900e+04,57969.000000
mean,17.579136,17.597864,17.897729,17.303203,7.073019e+06,0.168178
std,37.563762,37.599880,38.223225,37.032695,2.084862e+07,13.656731
min,0.097000,0.089000,0.102000,0.089000,0.000000e+00,-97.440000
25%,2.970000,2.980000,3.030000,2.920000,4.011100e+05,-1.190000
50%,6.940000,6.960000,7.070000,6.850000,1.480000e+06,0.000000
75%,16.210000,16.230000,16.590000,15.930000,5.110000e+06,1.190000
max,505.000000,505.000000,513.000000,502.010000,6.071300e+08,2696.000000


In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57969 entries, 0 to 57968
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Company   57969 non-null  object 
 1   Date      57969 non-null  object 
 2   Price     57969 non-null  float64
 3   Open      57969 non-null  float64
 4   High      57969 non-null  float64
 5   Low       57969 non-null  float64
 6   Vol.      57949 non-null  float64
 7   Change %  57969 non-null  float64
dtypes: float64(6), object(2)
memory usage: 3.5+ MB


In [5]:
ds.columns = ds.columns.str.lower()

In [6]:
print(ds["company"].unique())
ds["company"].nunique()

['Abu Dhabi Islamic Bank' 'Abu Qir Fertilizers and Chemical Industries'
 'Alexandria Mineral Oils' 'Arabian Cement Co SAE' 'Beltone Financial Hld'
 'Commercial Int Bank' 'EFG Hermes Holdings' 'Eastern Tobacco'
 'Edita Food' 'Egypt Aluminum' 'Emaar Misr for Development SAE' 'GB AUTO'
 'Ibnsina Pharma' 'Juhayna Food' 'Kima' 'Misr Cement'
 'Misr El Gadida for Housing Dev' 'Orascom Construction Ltd'
 'Orascom Hotels' 'Orascom Invest' 'Oriental Weavers' 'Palm Hills Develop'
 'Qalaa Holdings' 'Raya Holding' 'T M G Holding' 'Telecom Egypt'
 'Valmore Holding' 'Fawry Banking and Payment' 'Tenth of Ramadan'
 'Valmore Holding A' 'E-finance']


31

In [8]:
ds.head()

,company,date,price,open,high,low,vol.,change %
0,Abu Dhabi Islamic Bank,2018-01-02,5.400,5.390,5.45,5.380,282800.0,0.26
1,Abu Qir Fertilizers and Chemical Industries,2018-01-02,21.750,21.750,22.17,21.420,286920.0,1.16
2,Alexandria Mineral Oils,2018-01-02,9.210,9.250,9.37,9.100,1890000.0,-0.43
3,Arabian Cement Co SAE,2018-01-02,7.710,7.700,7.81,7.680,197770.0,1.05
4,Beltone Financial Hld,2018-01-02,4.158,4.154,4.20,4.122,292830.0,0.10
